<h2><center> RPS - Leaderboard Race Animation </center></h2>

<h2><center> <img src="https://media.giphy.com/media/6Z3D5t31ZdoNW/giphy.gif" alt="Run img"></center></h2>


## About this Notebook

The purpose of this notebook is just for fun :D

The coding is inspired by this [article](https://www.dunderdata.com/blog/create-a-bar-chart-race-animation-in-python-with-matplotlib).

I'll keep updating this notebook.

## Updates

**Version 20**: added individual animation for gold, silver and bronze battlegrounds

**Version 23**: data I collected before Feb 4th, 2021 has too much gap so I filtered them. Now data range is from Feb 4th, 2021 and the frequency is 2 hours

**Version 28**: animation to jshtml has duration limitation so I switched to .mp4 video

**Version 45**: improved the runtime by sacrificing animation smoothness 

In [ ]:
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime, timezone
from IPython.display import HTML, Video
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [ ]:
leaderboard_df = pd.read_csv("../input/rpssanta20-leaderboard/rps_leaderboard.csv") 
leaderboard_temp = leaderboard_df[leaderboard_df['time']== leaderboard_df['time'].max()].reset_index(drop=True)

In [ ]:
leaderboard_df.head()

In [ ]:
### Reshape dataset
df = pd.DataFrame(columns=leaderboard_df['team_id'].sort_values().unique(), index=leaderboard_df['time'].sort_values().unique())
for t in leaderboard_df['time'].unique():
    df.loc[t] = leaderboard_df[leaderboard_df['time'] == t].sort_values('team_id')['score'].tolist()
df = df.astype('float')
df = df.reset_index().rename(columns = {'index':'date'})

In [ ]:
df.head()

In [ ]:
top = 250
df = df[['date'] + leaderboard_temp['team_id'][:top].tolist()]
df = df[df['date'] > 1612431713].reset_index(drop=True)

In [ ]:
def prepare_data(df, steps=10):
    ### clean data
    df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x, tz=timezone.utc).strftime("%B %d, %Y %H:%M"))
    df = df.rename(columns = dict(zip(leaderboard_temp['team_id'], leaderboard_temp['team_name'])))
    df = df.reset_index(drop = True)
    
    ### prepare for smooth animation
    df.index = df.index * steps
    last_idx = df.index[-1] + 1
    df_expanded = df.reindex(range(last_idx))
    df_expanded['date'] = df_expanded['date'].fillna(method = 'ffill')
    df_expanded = df_expanded.set_index('date')
    df_rank_expanded = df_expanded.rank(axis = 1, method = 'first')
    df_expanded = df_expanded.interpolate()
    df_rank_expanded = df_rank_expanded.interpolate()
    return df_expanded, df_rank_expanded

df_expanded, df_rank_expanded = prepare_data(df)
df_expanded.head()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# plt.rcParams["mathtext.fontset"] = "cm"

colors = plt.cm.tab20(range(20))

<h2 style="color:gold">GOLD ZONE BATTLEGROUNDS</h2>.

In [ ]:
def nice_axes(ax):
    ax.set_facecolor('.92')
    ax.tick_params(labelsize=8, length=0)
    # ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]
    
def init():
    ax.clear()
    nice_axes(ax)

def update(i):
    for bar in ax.containers:
        bar.remove()
    for t in ax.texts:
        t.set_visible(False)
    y = df_rank_expanded.iloc[i]
    width = df_expanded.iloc[i]
    labels = df_expanded.columns
    ax.barh(y=y, width=width, color=colors, tick_label=labels)
    date_str = df_expanded.index[i] + ' UTC'
    ax.set_title(date_str, fontsize=12)
    ax.set_xlim(leaderboard_df['score'][50] - 100, max(leaderboard_df['score']) + 50)
    ax.set_ylim(top - 50.5, top + 0.5)
    for i, v in enumerate(width):
        ax.text(v + 3, y[i] - 0.2, str(round(v,1)), fontsize = 6, clip_on=True)
    if top > 13:
        ax.axhline(top - 12.5, color="gold")
        ax.text(max(leaderboard_df['score']) - 10, top - 12.3, 'gold line', color = 'gold')
    if top > 83:
        ax.axhline(top - 82.5, color="silver")
        ax.text(1100, top - 82.3, 'silver line', color = 'silver')
    if top > 166:
        ax.axhline(top - 165.5, color="#a97142")
        ax.text(1100, top - 165.3, 'bronze line', color = '#a97142') 
        
fig = plt.Figure(figsize=(8, 12), dpi = 111)
ax = fig.add_subplot()

anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(df_expanded), 
                     interval=75, repeat=False, cache_frame_data = False)

anim.save('gold.mp4')
Video("./gold.mp4", embed=True)

<h2 style="color:silver">SILVER ZONE BATTLEGROUNDS</h2>.

In [ ]:
def update(i):
    for bar in ax.containers:
        bar.remove()
    for t in ax.texts:
        t.set_visible(False)
    y = df_rank_expanded.iloc[i]
    width = df_expanded.iloc[i]
    labels = df_expanded.columns
    ax.barh(y=y, width=width, color=colors, tick_label=labels)
    date_str = df_expanded.index[i] + ' UTC'
    ax.set_title(date_str, fontsize=12)
    ax.set_xlim(leaderboard_df['score'][100] - 100, leaderboard_df['score'][50] + 50)
    ax.set_ylim(top - 100.5, top - 50.5)
    for i, v in enumerate(width):
        ax.text(v + 3, y[i] - 0.2, str(round(v,1)), fontsize = 6, clip_on=True)
    if top > 13:
        ax.axhline(top - 12.5, color="gold")
        ax.text(1100, top - 12.3, 'gold line', color = 'gold')
    if top > 83:
        ax.axhline(top - 82.5, color="silver")
        ax.text(leaderboard_df['score'][50] + 10, top - 82.3, 'silver line', color = 'silver')
    if top > 166:
        ax.axhline(top - 165.5, color="#a97142")
        ax.text(960, top - 165.3, 'bronze line', color = '#a97142') 
        
fig = plt.Figure(figsize=(8, 12), dpi = 111)
ax = fig.add_subplot()

anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(df_expanded), 
                     interval=75, repeat=False, cache_frame_data = False)

anim.save('silver.mp4')
Video("./silver.mp4", embed=True)

<h2 style="color:#a97142">BRONZE ZONE BATTLEGROUNDS</h2>.

In [ ]:

def update(i):
    for bar in ax.containers:
        bar.remove()
    for t in ax.texts:
        t.set_visible(False)
    y = df_rank_expanded.iloc[i]
    width = df_expanded.iloc[i]
    labels = df_expanded.columns
    ax.barh(y=y, width=width, color=colors, tick_label=labels)
    date_str = df_expanded.index[i] + ' UTC'
    ax.set_title(date_str, fontsize=12)
    ax.set_xlim(leaderboard_df['score'][190] - 50, leaderboard_df['score'][140] + 50)
    ax.set_ylim(top - 190.5, top - 140.5)
    for i, v in enumerate(width):
        ax.text(v + 3, y[i] - 0.2, str(round(v,1)), fontsize = 6, clip_on=True)
    if top > 13:
        ax.axhline(top - 12.5, color="gold")
        ax.text(1100, top - 12.3, 'gold line', color = 'gold')
    if top > 83:
        ax.axhline(top - 82.5, color="silver")
        ax.text(1000, top - 82.3, 'silver line', color = 'silver')
    if top > 166:
        ax.axhline(top - 165.5, color="#a97142")
        ax.text(leaderboard_df['score'][140] + 10, top - 165.3, 'bronze line', color = '#a97142') 
        
fig = plt.Figure(figsize=(8, 12), dpi = 111)
ax = fig.add_subplot()

anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(df_expanded), 
                     interval=75, repeat=False, cache_frame_data = False)

anim.save('bronze.mp4')
Video("./bronze.mp4", embed=True)